In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1168859,2021-04-10,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1168860,2021-04-11,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1168861,2021-04-12,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1168862,2021-04-13,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
31347,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
31349,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
31351,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
31353,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
31355,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
720525,2021-04-10,Arkansas,Arkansas,5001,2085,36.00,5000,Arkansas,AR,Arkansas,State,3017804
720527,2021-04-11,Arkansas,Arkansas,5001,2085,36.00,5000,Arkansas,AR,Arkansas,State,3017804
720529,2021-04-12,Arkansas,Arkansas,5001,2086,36.00,5000,Arkansas,AR,Arkansas,State,3017804
720531,2021-04-13,Arkansas,Arkansas,5001,2085,36.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1168859,2021-04-10,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1168860,2021-04-11,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1168861,2021-04-12,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1168862,2021-04-13,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-04-14') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
449,2021-04-14,Snohomish,Washington,34114,564.00,53061,WA,County,822083,4149.70,68.61
896,2021-04-14,Cook,Illinois,516399,10347.00,17031,IL,County,5150233,10026.71,200.90
1342,2021-04-14,Orange,California,268521,4862.00,6059,CA,County,3175692,8455.51,153.10
1787,2021-04-14,Maricopa,Arizona,529471,9741.00,4013,AZ,County,4485414,11804.28,217.17
2232,2021-04-14,Los Angeles,California,1227626,23553.00,6037,CA,County,10039107,12228.44,234.61
...,...,...,...,...,...,...,...,...,...,...,...
1168251,2021-04-14,Wheeler,Oregon,25,1.00,41069,OR,County,1332,1876.88,75.08
1168435,2021-04-14,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00
1168588,2021-04-14,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00
1168737,2021-04-14,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
449,2021-04-14,Snohomish,Washington,34114,564.00,53061,WA,County,822083,4149.70,68.61,68.61,4149.70
896,2021-04-14,Cook,Illinois,516399,10347.00,17031,IL,County,5150233,10026.71,200.90,200.90,10026.71
1342,2021-04-14,Orange,California,268521,4862.00,6059,CA,County,3175692,8455.51,153.10,153.10,8455.51
1787,2021-04-14,Maricopa,Arizona,529471,9741.00,4013,AZ,County,4485414,11804.28,217.17,217.17,11804.28
2232,2021-04-14,Los Angeles,California,1227626,23553.00,6037,CA,County,10039107,12228.44,234.61,234.61,12228.44
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1168251,2021-04-14,Wheeler,Oregon,25,1.00,41069,OR,County,1332,1876.88,75.08,75.08,1876.88
1168435,2021-04-14,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00,0.00,4044.12
1168588,2021-04-14,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00,0.00,4352.81
1168737,2021-04-14,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00,0.00,591.72


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
753382,2021-04-14,Hale,Alabama,2186,75.00,1065,AL,County,14651,14920.48,511.91,511.91,14920.48,1
819977,2021-04-14,Clarke,Alabama,3465,60.00,1025,AL,County,23622,14668.53,254.00,254.00,14668.53,2
589581,2021-04-14,Lowndes,Alabama,1365,53.00,1085,AL,County,9726,14034.55,544.93,544.93,14034.55,3
438756,2021-04-14,Franklin,Alabama,4205,82.00,1059,AL,County,31362,13407.95,261.46,261.46,13407.95,4
522809,2021-04-14,Etowah,Alabama,13673,346.00,1055,AL,County,102268,13369.77,338.33,338.33,13369.77,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1108860,2021-04-14,Platte,Wyoming,615,11.00,56031,WY,County,8393,7327.53,131.06,131.06,7327.53,19
719004,2021-04-14,Converse,Wyoming,989,17.00,56009,WY,County,13822,7155.26,122.99,122.99,7155.26,20
893689,2021-04-14,Lincoln,Wyoming,1385,12.00,56023,WY,County,19830,6984.37,60.51,60.51,6984.37,21
994485,2021-04-14,Niobrara,Wyoming,150,2.00,56027,WY,County,2356,6366.72,84.89,84.89,6366.72,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
589581,2021-04-14,Lowndes,Alabama,1365,53.00,1085,AL,County,9726,14034.55,544.93,544.93,14034.55,3,1
753382,2021-04-14,Hale,Alabama,2186,75.00,1065,AL,County,14651,14920.48,511.91,511.91,14920.48,1,2
250886,2021-04-14,Walker,Alabama,7101,275.00,1127,AL,County,63521,11178.98,432.93,432.93,11178.98,28,3
635778,2021-04-14,Greene,Alabama,910,34.00,1063,AL,County,8111,11219.33,419.18,419.18,11219.33,27,4
521265,2021-04-14,Clay,Alabama,1495,55.00,1027,AL,County,13235,11295.81,415.56,415.56,11295.81,24,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
634241,2021-04-14,Hot Springs,Wyoming,363,3.00,56017,WY,County,4413,8225.70,67.98,67.98,8225.70,17,19
893689,2021-04-14,Lincoln,Wyoming,1385,12.00,56023,WY,County,19830,6984.37,60.51,60.51,6984.37,21,20
871446,2021-04-14,Uinta,Wyoming,2153,12.00,56041,WY,County,20226,10644.71,59.33,59.33,10644.71,4,21
249710,2021-04-14,Teton,Wyoming,3684,9.00,56039,WY,County,23464,15700.65,38.36,38.36,15700.65,1,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,68.61,4149.70,19,22
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,68.61,4149.70,19,22
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,68.61,4149.70,19,22
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,68.61,4149.70,19,22
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,68.61,4149.70,19,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1166115,2021-04-10,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1166116,2021-04-11,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1166117,2021-04-12,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1166118,2021-04-13,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
750258,2020-03-30,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,511.91,14920.48,2,1,1.00
750259,2020-03-31,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,511.91,14920.48,2,1,0.00
750260,2020-04-01,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,511.91,14920.48,2,1,0.00
750261,2020-04-02,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,511.91,14920.48,2,1,0.00
750262,2020-04-03,Hale,Alabama,3,0.00,1065,AL,County,14651,20.48,0.00,511.91,14920.48,2,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
908613,2021-04-10,Crook,Wyoming,425,12.00,56011,WY,County,7584,5603.90,158.23,158.23,5643.46,6,23,0.00
908614,2021-04-11,Crook,Wyoming,425,12.00,56011,WY,County,7584,5603.90,158.23,158.23,5643.46,6,23,0.00
908615,2021-04-12,Crook,Wyoming,425,12.00,56011,WY,County,7584,5603.90,158.23,158.23,5643.46,6,23,0.00
908616,2021-04-13,Crook,Wyoming,428,12.00,56011,WY,County,7584,5643.46,158.23,158.23,5643.46,6,23,3.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
587219,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14034.55,1,3,1.00,0.00
587220,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14034.55,1,3,0.00,0.00
587221,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14034.55,1,3,0.00,0.00
587222,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14034.55,1,3,0.00,0.00
587223,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14034.55,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
585289,2021-04-10,Albany,Wyoming,4055,11.00,56001,WY,County,38880,10429.53,28.29,28.29,10534.98,23,5,0.00,0.00
585290,2021-04-11,Albany,Wyoming,4055,11.00,56001,WY,County,38880,10429.53,28.29,28.29,10534.98,23,5,0.00,0.00
585291,2021-04-12,Albany,Wyoming,4079,11.00,56001,WY,County,38880,10491.26,28.29,28.29,10534.98,23,5,24.00,0.00
585292,2021-04-13,Albany,Wyoming,4093,11.00,56001,WY,County,38880,10527.26,28.29,28.29,10534.98,23,5,14.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-04-14') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
299462,2021-04-14,Imperial,California,27887,717.00,6025,CA,County,181215,15388.90,395.66,395.66,15388.90,1,2,36.00,1.00
2232,2021-04-14,Los Angeles,California,1227626,23553.00,6037,CA,County,10039107,12228.44,234.61,234.61,12228.44,2,5,370.00,48.00
149176,2021-04-14,San Bernardino,California,294101,4521.00,6071,CA,County,2180085,13490.35,207.38,207.38,13490.35,3,4,176.00,28.00
524305,2021-04-14,Inyo,California,1391,36.00,6027,CA,County,18039,7711.07,199.57,199.57,7711.07,4,25,1.00,0.00
66737,2021-04-14,Stanislaus,California,60129,1023.00,6099,CA,County,550660,10919.44,185.78,185.78,10919.44,5,9,46.00,3.00
30942,2021-04-14,Riverside,California,296908,4514.00,6065,CA,County,2470546,12017.91,182.71,182.71,12017.91,6,6,107.00,10.00
82734,2021-04-14,Tulare,California,49318,825.00,6107,CA,County,466195,10578.84,176.96,176.96,10578.84,7,10,18.00,1.00
53105,2021-04-14,San Joaquin,California,71229,1334.00,6077,CA,County,762148,9345.82,175.03,175.03,9345.82,8,19,51.00,0.00
30134,2021-04-14,Fresno,California,100357,1636.00,6019,CA,County,999101,10044.73,163.75,163.75,10044.73,9,14,80.00,3.00
475173,2021-04-14,Merced,California,31128,450.00,6047,CA,County,277680,11210.03,162.06,162.06,11210.03,10,8,23.00,1.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
1108411,2021-04-14,Lassen,California,5666,1.00,6035,CA,County,30573,18532.69,3.27,3.27,18532.69,56,1,0.00,0.00
299462,2021-04-14,Imperial,California,27887,717.00,6025,CA,County,181215,15388.90,395.66,395.66,15388.90,1,2,36.00,1.00
635720,2021-04-14,Kings,California,22730,245.00,6031,CA,County,152940,14862.04,160.19,160.19,14862.04,11,3,7.00,0.00
149176,2021-04-14,San Bernardino,California,294101,4521.00,6071,CA,County,2180085,13490.35,207.38,207.38,13490.35,3,4,176.00,28.00
2232,2021-04-14,Los Angeles,California,1227626,23553.00,6037,CA,County,10039107,12228.44,234.61,234.61,12228.44,2,5,370.00,48.00
30942,2021-04-14,Riverside,California,296908,4514.00,6065,CA,County,2470546,12017.91,182.71,182.71,12017.91,6,6,107.00,10.00
187533,2021-04-14,Kern,California,107591,1306.00,6029,CA,County,900202,11951.87,145.08,145.08,11951.87,14,7,157.00,0.00
475173,2021-04-14,Merced,California,31128,450.00,6047,CA,County,277680,11210.03,162.06,162.06,11210.03,10,8,23.00,1.00
66737,2021-04-14,Stanislaus,California,60129,1023.00,6099,CA,County,550660,10919.44,185.78,185.78,10919.44,5,9,46.00,3.00
82734,2021-04-14,Tulare,California,49318,825.00,6107,CA,County,466195,10578.84,176.96,176.96,10578.84,7,10,18.00,1.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
299462,2021-04-14,Imperial,California,27887,717.00,6025,CA,County,181215,15388.90,395.66,395.66,15388.90,1,2,36.00,1.00
2232,2021-04-14,Los Angeles,California,1227626,23553.00,6037,CA,County,10039107,12228.44,234.61,234.61,12228.44,2,5,370.00,48.00
149176,2021-04-14,San Bernardino,California,294101,4521.00,6071,CA,County,2180085,13490.35,207.38,207.38,13490.35,3,4,176.00,28.00
524305,2021-04-14,Inyo,California,1391,36.00,6027,CA,County,18039,7711.07,199.57,199.57,7711.07,4,25,1.00,0.00
66737,2021-04-14,Stanislaus,California,60129,1023.00,6099,CA,County,550660,10919.44,185.78,185.78,10919.44,5,9,46.00,3.00
30942,2021-04-14,Riverside,California,296908,4514.00,6065,CA,County,2470546,12017.91,182.71,182.71,12017.91,6,6,107.00,10.00
82734,2021-04-14,Tulare,California,49318,825.00,6107,CA,County,466195,10578.84,176.96,176.96,10578.84,7,10,18.00,1.00
53105,2021-04-14,San Joaquin,California,71229,1334.00,6077,CA,County,762148,9345.82,175.03,175.03,9345.82,8,19,51.00,0.00
30134,2021-04-14,Fresno,California,100357,1636.00,6019,CA,County,999101,10044.73,163.75,163.75,10044.73,9,14,80.00,3.00
475173,2021-04-14,Merced,California,31128,450.00,6047,CA,County,277680,11210.03,162.06,162.06,11210.03,10,8,23.00,1.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,395.66,15388.90,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,395.66,15388.90,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,395.66,15388.90,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,395.66,15388.90,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,395.66,15388.90,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
3922,04/10/21,Lassen,5660,1.00,18513.07,3.27,3.27,18532.69,56,1,0.00,0.00
3923,04/11/21,Lassen,5660,1.00,18513.07,3.27,3.27,18532.69,56,1,0.00,0.00
3924,04/12/21,Lassen,5660,1.00,18513.07,3.27,3.27,18532.69,56,1,0.00,0.00
3925,04/13/21,Lassen,5666,1.00,18532.69,3.27,3.27,18532.69,56,1,6.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,395.66,15388.90,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,395.66,15388.90,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,395.66,15388.90,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,395.66,15388.90,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,395.66,15388.90,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
4008,04/10/21,Merced,31009,448.00,11167.17,161.34,162.06,11210.03,10,8,0.00,0.00
4009,04/11/21,Merced,31009,448.00,11167.17,161.34,162.06,11210.03,10,8,0.00,0.00
4010,04/12/21,Merced,31070,448.00,11189.14,161.34,162.06,11210.03,10,8,61.00,0.00
4011,04/13/21,Merced,31105,449.00,11201.74,161.70,162.06,11210.03,10,8,35.00,1.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)